# Introduction
In this notebook a simple spellchecker is built. It checks whether all words in a provided sentence are correct based on the 'brown' corpus from nltk library. All necessary components for the spellchecker are developed from scratch: the candidate model, the language model and the spelling error model. The implemented spelling error model is based on improved Levenshtein distance that takes also into account keyboard misclicks.

# Load libraries
First we import necessary libraries.

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import brown

# Build vocabulary
To check whether provided word is correct and to build a language model we need a word corpus. Here we will be using the 'brown' corpus from nltk.

In [2]:
nltk.download('brown')
words = brown.words()
words = [word.lower() for word in words]
counts = Counter(words)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
vocab = list(counts.keys())
vocab_len = len(vocab)
vocab_len

49815

# Identify misspelled word
To identify misspelled words we will simply compare them with a vocabulary.

In [4]:
def identify_mis(sentence, vocab):
  """
  Function that identify misspelled words based on provided vocabulary.
  Input:
  sentence - sentence in a form of string
  vocab - list of words, vocabulary
  Output:
  mis - list of misspelled words with their position
  """
  sentence = sentence.lower()
  words = re.findall('\w+', sentence)
  return [[pos, word] for pos, word in enumerate(words) if word not in vocab]
mis = identify_mis('This is a trest.', vocab)
print(mis)

[[3, 'trest']]


# Candidate model
The candidate model provides all possible words combination based on simply char operations such as: delete, insert, replace and switch. We define a function for each of the operations.

In [5]:
def delete_l(word):
  """
  Function that make a list of words by deleting one letter from a provided word.
  Input:
  word - in string format
  Output:
  del_l - list of words
  """
  del_l = []
  split_l = []
  split_l = [(word[:i],word[i:]) for i in range(len(word))]
  del_l = [word[:i] + word[i+1:] for i in range(len(word))]
  return del_l

def insert_l(word):
  """
  Function that make a list of words by inserting one letter from alphabet
  into a provided word.
  Input:
  word - in string format
  Output:
  ins_l - list of words
  """
  letters = 'abcdefghijklmnopqrstuvwxyz'
  ins_l = []
  split_l = []
  for c in range(len(word)+1):
      split_l.append((word[0:c],word[c:]))
  ins_l = [ a + l + b for a,b in split_l for l in letters]  
  return ins_l

def replace_l(word):
  """
  Function that make a list of words by replacing one letter from alphabet
  with a letter in a provided word.
  Input:
  word - in string format
  Output:
  rep_l - list of words
  """
  letters = 'abcdefghijklmnopqrstuvwxyz'
  rep_l = []
  split_l = []
  for c in range(len(word)):
    split_l.append((word[0:c],word[c:]))
  rep_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
  rep_set=set(rep_l)    
  rep_set.remove(word)
  # turn the set back into a list and sort it, for easier viewing
  rep_l = sorted(list(rep_set))
  return rep_l

def switch_l(word, verbose=False):
  """
  Function that make a list of words by switch adjacent letters.
  Input:
  word - in string format
  Output:
  switch_l - list of words
  """
  switch_l = []
  split_l = []
  len_word=len(word)
  for c in range(len_word):
      split_l.append((word[:c],word[c:]))
  switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]     
  return switch_l

In [6]:
print("Delete: ", delete_l(mis[0][1]))
print("Insert: ", insert_l(mis[0][1]))
print("Replace: ", replace_l(mis[0][1]))
print("Switch: ", switch_l(mis[0][1]))

Delete:  ['rest', 'test', 'trst', 'tret', 'tres']
Insert:  ['atrest', 'btrest', 'ctrest', 'dtrest', 'etrest', 'ftrest', 'gtrest', 'htrest', 'itrest', 'jtrest', 'ktrest', 'ltrest', 'mtrest', 'ntrest', 'otrest', 'ptrest', 'qtrest', 'rtrest', 'strest', 'ttrest', 'utrest', 'vtrest', 'wtrest', 'xtrest', 'ytrest', 'ztrest', 'tarest', 'tbrest', 'tcrest', 'tdrest', 'terest', 'tfrest', 'tgrest', 'threst', 'tirest', 'tjrest', 'tkrest', 'tlrest', 'tmrest', 'tnrest', 'torest', 'tprest', 'tqrest', 'trrest', 'tsrest', 'ttrest', 'turest', 'tvrest', 'twrest', 'txrest', 'tyrest', 'tzrest', 'traest', 'trbest', 'trcest', 'trdest', 'treest', 'trfest', 'trgest', 'trhest', 'triest', 'trjest', 'trkest', 'trlest', 'trmest', 'trnest', 'troest', 'trpest', 'trqest', 'trrest', 'trsest', 'trtest', 'truest', 'trvest', 'trwest', 'trxest', 'tryest', 'trzest', 'treast', 'trebst', 'trecst', 'tredst', 'treest', 'trefst', 'tregst', 'trehst', 'treist', 'trejst', 'trekst', 'trelst', 'tremst', 'trenst', 'treost', 'trepst', 

We also make a function that will build a set of all possible words by doing a single and a double edit.

In [7]:
def edit_one_l(word):  
  """
  Makes a set of all possible words that are one edit away.
  Input:
  word - in string format
  Output:
  edit_one_set - set of possible words
  """
  edit_one_set = set()
  edit_one_set.update(delete_l(word))
  edit_one_set.update(switch_l(word))
  edit_one_set.update(replace_l(word))
  edit_one_set.update(insert_l(word))
  return edit_one_set

def edit_two_l(word):
  """
  Makes a set of all possible words that are two edits away.
  Input:
  word - in string format
  Output:
  edit_one_set - set of possible words
  """
  edit_two_set = set()
  edit_one = edit_one_l(word)
  for w in edit_one:
    if w:
      edit_two = edit_one_l(w)
      edit_two_set.update(edit_two)
  return edit_two_set

In [8]:
edit_1 = edit_one_l(mis[0][1])
edit_2 = edit_two_l(mis[0][1])
print("One edit: ", edit_1)
print("Two edits: ", edit_2)

One edit:  {'erest', 'tlest', 'gtrest', 'mrest', 'trbst', 'tress', 'tnrest', 'tresq', 'tjrest', 'trebst', 'trqest', 'trect', 'treust', 'trxst', 'tres', 'tresr', 'trrest', 'trestx', 'trets', 'tzest', 'tkest', 'tresx', 'treset', 'tcest', 'ltrest', 'tnest', 'htrest', 'trsest', 'trefst', 'tirest', 'trert', 'btrest', 'grest', 'trset', 'tvrest', 'tregt', 'trpst', 'trnst', 'tzrest', 'triest', 'trenst', 'wtrest', 'trzest', 'tfrest', 'treslt', 'tjest', 'treqst', 'trestq', 'krest', 'txest', 'tsrest', 'trnest', 'tresvt', 'trestl', 'dtrest', 'lrest', 'qrest', 'tresjt', 'trestr', 'trests', 'trzst', 'trestj', 'tresth', 'trejst', 'frest', 'treost', 'tredst', 'trbest', 'trmest', 'treft', 'teest', 'trebt', 'vrest', 'tresn', 'tretst', 'tresnt', 'rrest', 'tresut', 'treso', 'ktrest', 'trese', 'tresh', 'wrest', 'mtrest', 'trcest', 'trjest', 'trewst', 'trestt', 'treht', 'tbest', 'xtrest', 'tkrest', 'tresbt', 'tresct', 'trtest', 'tarest', 'treyst', 'crest', 'torest', 'tresty', 'orest', 'tresht', 'treit', 'tr

# Language model
To assess proposed words a laguage model is used. It will simply return n the most probable words from our set of possible words.

In [9]:
def make_probs(counts):
  """
  Makes a dict of words probabilities (unigram based)
  Input:
  word - counts of words in a corpus
  Output:
  prob - dict of probabilities of words
  """
  probs = {}  # return this variable correctly
  total = sum(counts.values())
  for key in counts.keys():
    probs[key] = counts[key] / total
  return probs
probs = make_probs(counts)
print('Probability of "the" {:.2f}%'.format(probs['the']*100))

Probability of "the" 6.03%


In [10]:
def get_suggestions(word, probs, vocab, n=2, return_probs=False):
  """
  Proposes n the most probable words
  Input:
  word - a misspelled word
  probs - dictionary of word probabilities
  n - expected number of the most probable words
  return_probs - switch to append word probabilities
  Output:
  n_best - list of n the most probable words
  """
  suggestions = []
  n_best = []
  suggestions = list((word in vocab and word) or edit_one_l(word).intersection(vocab) or edit_two_l(word).intersection(vocab))
  if return_probs:
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))][:n]
  else:
    n_best = [s for s in list(reversed(suggestions))][:n]
  suggestions = set(suggestions)
  return n_best

In [11]:
n = 4
sugg = get_suggestions(mis[0][1], probs, vocab, n, return_probs=True)
print('{} suggestions for word "{}":'.format(n, mis[0][1]))
for x in range(n):
  print("{}: {:.6f}%".format(sugg[x][0], sugg[x][1]*100))

4 suggestions for word "trest":
crest: 0.001033%
truest: 0.000172%
test: 0.010248%
trust: 0.004478%


# Spelling Error model
Finally, we build a spelling error model that will tell us which of the selected words is the most likely to be the word that we are looking for. For that we will use the Levenshtein distance function but we improve it. It will take also into account misclicks on a QWERTY keyboard.  
First, we build a typical function for the Levenshtein distance.

In [12]:
def min_dist(source, target):
  """
  Calculate Levenshtein distance between source and target word.
  Input:
  source - a source word
  target - a target word
  Output:
  D - minimum distance
  m - Levenshtein distance matrix
  """
  ins_weight = 1
  del_weight = 1
  rep_weight = 2
  # use deletion and insert cost as  1
  m = len(source) 
  n = len(target) 
  #initialize cost matrix with zeros and dimensions (m+1,n+1) 
  D = np.zeros((m+1, n+1), dtype=int) 
  for row in range(1,m+1): # Replace None with the proper range
    D[row,0] = D[row-1,0] + del_weight
  for col in range(1,n+1): # Replace None with the proper range
    D[0,col] = D[0,col-1] + ins_weight
  for row in range(1,m+1): 
    for col in range(1,n+1):
      if source[row-1] == target[col-1]:
        r_weight = 0
      else:
        r_weight = rep_weight
      D[row,col] = min([D[row-1,col]+del_weight, D[row,col-1]+ins_weight, D[row-1,col-1]+r_weight])       
  med = D[m,n]
  return D, med

We want to improve the Levenshtein distance by incoporating simple misclicks on the QWERTY keyboard. We want to treat as miclicks only keys that are in a vicinity of the clicked key. Therefore we build a QWERTY key-distance function based on the l2-norm.

In [13]:
def make_qwerty():
  """
  Makes dict with keys position.
  """
  r0 =  ['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=']
  r0s = ['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
  r1 =  ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\']
  r1s = ['{', '}', '|']
  r2 =  ['a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'"]
  r2s = [':', '"']
  r3 =  ['z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/']
  r3s = ['<', '>', '?']
  char_pos = {}
  for i, key in enumerate(r0):
    char_pos[key] = np.array([0., i])
  for i, key in enumerate(r0s):
    char_pos[key] = np.array([0., i])
  for i, key in enumerate(r1):
    char_pos[key] = np.array([1., i+1.5])
  for i, key in enumerate(r1s):
    char_pos[key] = np.array([1., i+11.5])
  for i, key in enumerate(r2):
    char_pos[key] = np.array([2., i+1.83])
  for i, key in enumerate(r2s):
    char_pos[key] = np.array([2., i+10.83])
  for i, key in enumerate(r3):
    char_pos[key] = np.array([3., i+2.4])
  for i, key in enumerate(r3s):
    char_pos[key] = np.array([3., i+9.4])
  return char_pos

char_pos = make_qwerty()

def calc_dist(ch1, ch2, char_pos, weight=0.8):
  """
  Calculates weighted l2 distance between keys.
  """
  return np.linalg.norm(char_pos[ch1]-char_pos[ch2])*weight

The weight in calc_dist has been chosen based on a trial-and-error approach. If the distance is above  1 it becomes more beneficial to insert or delete a character, so we want only the keys that are around to have distance below 1.

In [14]:
print("Distance from 'f' to 'e' is: {:.2f}".format(calc_dist('f','e',char_pos)))
print("Distance from 'f' to 'r' is: {:.2f}".format(calc_dist('f','r',char_pos)))
print("Distance from 'f' to 't' is: {:.2f}".format(calc_dist('f','t',char_pos)))
print("Distance from 'f' to 'd' is: {:.2f}".format(calc_dist('f','d',char_pos)))
print("Distance from 'f' to 'g' is: {:.2f}".format(calc_dist('f','g',char_pos)))
print("Distance from 'f' to 'c' is: {:.2f}".format(calc_dist('f','c',char_pos)))
print("Distance from 'f' to 'v' is: {:.2f}".format(calc_dist('f','v',char_pos)))
print("Distance from 'f' to 'b' is: {:.2f}".format(calc_dist('f','b',char_pos)))
print("Distance from 'f' to ']' is: {:.2f}".format(calc_dist('z',']',char_pos)))

Distance from 'f' to 'e' is: 1.33
Distance from 'f' to 'r' is: 0.84
Distance from 'f' to 't' is: 0.96
Distance from 'f' to 'd' is: 0.80
Distance from 'f' to 'g' is: 0.80
Distance from 'f' to 'c' is: 0.87
Distance from 'f' to 'v' is: 0.92
Distance from 'f' to 'b' is: 1.49
Distance from 'f' to ']' is: 8.24


We define a modified Levenshtein distance function by making the replacement cost dependent on keys distance with a cap set on 2.

In [15]:
def mod_min_dist(source, target, char_pos, return_D=False):
  """
  Calculate improved Levenshtein distance between source and target word.
  It takes into account misclicks on a QWERTY keyboard.
  Input:
  source - a source word
  target - a target word
  Output:
  D - minimum distance
  m - Levenshtein distance matrix
  """
  # use deletion and insert cost as  1
  ins_weight = 1
  del_weight = 1
  m = len(source) 
  n = len(target) 
  #initialize cost matrix with zeros and dimensions (m+1,n+1) 
  D = np.zeros((m+1, n+1), dtype=float) 
  for row in range(1,m+1):
    D[row,0] = D[row-1,0] + del_weight
  for col in range(1,n+1):
    D[0,col] = D[0,col-1] + ins_weight    
  # fill the rest
  for row in range(1,m+1): 
    for col in range(1,n+1):
      if source[row-1] == target[col-1]:
        r_weight = 0
      else:
        # we put a cap of 2 on min distance to not exceed default Levenshtein value
        r_weight = min(calc_dist(source[row-1], target[col-1], char_pos), 2)
      D[row,col] = min([D[row-1,col]+del_weight, D[row,col-1]+ins_weight, D[row-1,col-1]+r_weight])
  med = D[m,n]
  if return_D:
    return D, med
  else:
    return med

Now let's test how it works.

In [16]:
source =  'eer'
target = 'near'
print('SOURCE: "{}", TARGER: "{}"'.format(source, target))
print('regular')
matrix, min_edits = min_dist(source, target)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)
print('modified')
matrix, min_edits = mod_min_dist(source, target, char_pos, return_D=True)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)

SOURCE: "eer", TARGER: "near"
regular
minimum edits:  3
   0  1  2  3  4
0  0  1  2  3  4
1  1  2  1  2  3
2  2  3  2  3  4
3  3  4  3  4  3
modified
minimum edits:  2.5572077574941634
     0    1    2         3         4
0  0.0  1.0  2.0  3.000000  4.000000
1  1.0  2.0  1.0  2.000000  3.000000
2  2.0  3.0  2.0  2.557208  2.800000
3  3.0  4.0  3.0  3.557208  2.557208


From 'eer' to 'near' we have now 1 insertion and 1 far replace.

In [17]:
source =  'chaur'
target = 'chair'
print('SOURCE: "{}", TARGER: "{}"'.format(source, target))
print('regular')
matrix, min_edits = min_dist(source, target)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)
print('modified')
matrix, min_edits = mod_min_dist(source, target, char_pos, return_D=True)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)

SOURCE: "chaur", TARGER: "chair"
regular
minimum edits:  2
   0  1  2  3  4  5
0  0  1  2  3  4  5
1  1  0  1  2  3  4
2  2  1  0  1  2  3
3  3  2  1  0  1  2
4  4  3  2  1  2  3
5  5  4  3  2  3  2
modified
minimum edits:  0.8
     0    1    2    3    4    5
0  0.0  1.0  2.0  3.0  4.0  5.0
1  1.0  0.0  1.0  2.0  3.0  4.0
2  2.0  1.0  0.0  1.0  2.0  3.0
3  3.0  2.0  1.0  0.0  1.0  2.0
4  4.0  3.0  2.0  1.0  0.8  1.8
5  5.0  4.0  3.0  2.0  1.8  0.8


'chaur' to 'chair' is one replace away.

In [18]:
source =  'chair'
target = 'chatr'
print('SOURCE: "{}", TARGER: "{}"'.format(source, target))
print('regular')
matrix, min_edits = min_dist(source, target)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)
print('modified')
matrix, min_edits = mod_min_dist(source, target, char_pos, return_D=True)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)

SOURCE: "chair", TARGER: "chatr"
regular
minimum edits:  2
   0  1  2  3  4  5
0  0  1  2  3  4  5
1  1  0  1  2  3  4
2  2  1  0  1  2  3
3  3  2  1  0  1  2
4  4  3  2  1  2  3
5  5  4  3  2  3  2
modified
minimum edits:  2.0
     0    1    2    3    4    5
0  0.0  1.0  2.0  3.0  4.0  5.0
1  1.0  0.0  1.0  2.0  3.0  4.0
2  2.0  1.0  0.0  1.0  2.0  3.0
3  3.0  2.0  1.0  0.0  1.0  2.0
4  4.0  3.0  2.0  1.0  2.0  3.0
5  5.0  4.0  3.0  2.0  1.8  2.0


But 'chatr' is far enough to activate our cap set on 2.0.

In [19]:
source =  'prroixmity'
target = 'proximity'
print('SOURCE: "{}", TARGER: "{}"'.format(source, target))
print('regular')
matrix, min_edits = min_dist(source, target)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)
print('modified')
matrix, min_edits = mod_min_dist(source, target, char_pos, return_D=True)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)

SOURCE: "prroixmity", TARGER: "proximity"
regular
minimum edits:  3
     0  1  2  3  4  5  6  7  8  9
0    0  1  2  3  4  5  6  7  8  9
1    1  0  1  2  3  4  5  6  7  8
2    2  1  0  1  2  3  4  5  6  7
3    3  2  1  2  3  4  5  6  7  8
4    4  3  2  1  2  3  4  5  6  7
5    5  4  3  2  3  2  3  4  5  6
6    6  5  4  3  2  3  4  5  6  7
7    7  6  5  4  3  4  3  4  5  6
8    8  7  6  5  4  3  4  3  4  5
9    9  8  7  6  5  4  5  4  3  4
10  10  9  8  7  6  5  6  5  4  3
modified
minimum edits:  3.0
       0    1    2    3         4         5         6         7    8    9
0    0.0  1.0  2.0  3.0  4.000000  5.000000  6.000000  7.000000  8.0  9.0
1    1.0  0.0  1.0  2.0  3.000000  4.000000  5.000000  6.000000  7.0  8.0
2    2.0  1.0  0.0  1.0  2.000000  3.000000  4.000000  5.000000  6.0  7.0
3    3.0  2.0  1.0  2.0  2.826034  3.826034  4.826034  5.826034  5.8  6.8
4    4.0  3.0  2.0  1.0  2.000000  3.000000  4.000000  5.000000  6.0  7.0
5    5.0  4.0  3.0  2.0  3.000000  2.000000  3.0000

Here one extra letter does not influence our results.

In [20]:
source =  'proxumity'
target = 'proximity'
print('SOURCE: "{}", TARGER: "{}"'.format(source, target))
print('regular')
matrix, min_edits = min_dist(source, target)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)
print('modified')
matrix, min_edits = mod_min_dist(source, target, char_pos, return_D=True)
print("minimum edits: ",min_edits)
df = pd.DataFrame(matrix, index=list(source).insert(0, '#'), columns=list(target).insert(0, '#'))
print(df)

SOURCE: "proxumity", TARGER: "proximity"
regular
minimum edits:  2
   0  1  2  3  4  5  6  7  8  9
0  0  1  2  3  4  5  6  7  8  9
1  1  0  1  2  3  4  5  6  7  8
2  2  1  0  1  2  3  4  5  6  7
3  3  2  1  0  1  2  3  4  5  6
4  4  3  2  1  0  1  2  3  4  5
5  5  4  3  2  1  2  3  4  5  6
6  6  5  4  3  2  3  2  3  4  5
7  7  6  5  4  3  2  3  2  3  4
8  8  7  6  5  4  3  4  3  2  3
9  9  8  7  6  5  4  5  4  3  2
modified
minimum edits:  0.8
     0    1    2    3    4    5    6    7    8    9
0  0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
1  1.0  0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0
2  2.0  1.0  0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0
3  3.0  2.0  1.0  0.0  1.0  2.0  3.0  4.0  5.0  6.0
4  4.0  3.0  2.0  1.0  0.0  1.0  2.0  3.0  4.0  5.0
5  5.0  4.0  3.0  2.0  1.0  0.8  1.8  2.8  3.8  4.8
6  6.0  5.0  4.0  3.0  2.0  1.8  0.8  1.8  2.8  3.8
7  7.0  6.0  5.0  4.0  3.0  2.0  1.8  0.8  1.8  2.8
8  8.0  7.0  6.0  5.0  4.0  3.0  2.8  1.8  0.8  1.8
9  9.0  8.0  7.0  6.0  5.0  4.0 

But here it again works.

# Wrapping up
Now we build a function to incorporates all of the above functions and it proposes a corrected words with use of them.

In [21]:
def check_spelling(sentence, vocab, probs, char_pos, n=5, verify=False):
  """
  Identifies misspelled words in a sentence and proposes corrections.
  Input:
  sentence - a sentence to check
  vocab - corpus vocabulary
  probs - words probabilities
  char_pos - dictionary of keys positions
  n - number of outputed possible words from language model
  verify - switch for returning verification lists
  """
  mis_l = identify_mis(sentence, vocab)
  sugg_l = [[word[1], get_suggestions(word[1], probs, vocab, n=n)] for word in mis_l]
  dist_l = [[[mod_min_dist(word[0], sugg, char_pos, return_D=False)] for sugg in word[1]] for word in sugg_l]
  if verify:
    return sugg_l, dist_l
  else:
    corr = [sugg_l[i][1][np.argmin(dist)] for i, dist in enumerate(dist_l)]
    print('Suggestions:')
    for i, row in enumerate(mis_l):
      print("{} -> {}".format(row[1], corr[i]))


First see what words are investigated by the algorithm.

In [22]:
sugg, dist = check_spelling("I'm sitring in a chaur.", vocab, probs, char_pos, verify=True)
for i, row in enumerate(sugg):
  print('Mistake: {}'.format(row[0]))
  print('Suggestions:')
  print(list(zip(row[1], dist[i])))

Mistake: sitring
Suggestions:
[('string', [1.0]), ('sitting', [0.8])]
Mistake: chaur
Suggestions:
[('chair', [0.8]), ('char', [1.0])]


If modified distance wouldn't be implemented, 'string' and 'char' would be more probabe that out true solution. So the provided method clearly works.

In [23]:
check_spelling("I'm sitring in a chaur.", vocab, probs, char_pos, verify=False)

Suggestions:
sitring -> sitting
chaur -> chair


Let's see another sentence.

In [24]:
check_spelling("My back hyrts.", vocab, probs, char_pos, verify=False)

Suggestions:
hyrts -> hurts


How about one further misclick?

In [25]:
check_spelling("My back htrts.", vocab, probs, char_pos, verify=False)

Suggestions:
htrts -> hurts


In [26]:
check_spelling("My back herts.", vocab, probs, char_pos, verify=False)

Suggestions:
herts -> hertz


And one more?

When the keys are too far apart, they are no longer treated as misclicks and other words are proposed.

# Conclusions
We have implemented a standard method for a spell checking but we have extended it by implementation of misclick-dependent Levenshtein distance function that assignes less weight to keys that next to each other. This resulted in improved method that easily handles these typical mistakes.